In [1]:
import utils
from skrub import TableReport
from pathlib import Path
import pandas as pd

In [2]:
X, y, X_test = utils.get_and_process_data()

/Users/felix/Python_Data_Challenge-1/utils.py:48: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
  closest_value = non_nan_values.iloc[(non_nan_values - value).abs().argmin()]
/Users/felix/Python_Data_Challenge-1/utils.py:48: FutureWarning: The behavior of Series.argmax/argmin with skipna=False and NAs, or with all-NAs is deprecated. In a future version this will raise ValueError.
  closest_value = non_nan_values.iloc[(non_nan_values - value).abs().argmin()]
/Users/felix/Python_Data_Challenge-1/utils.py:197: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting val

In [3]:
TableReport(X)

Processing column  23 / 23


,,,,,,,,,,,,,,,,,,,,,,,


In [6]:
pipeline = utils.create_pipeline(X)

In [9]:
pipeline.fit(X,y)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  []),
                                                 ('num', StandardScaler(),
                                                  ['latitude', 'longitude',
                                                   'Pressure_Tendency_(hPa/3h)',
                                                   'Wind_Speed_(m/s)',
                                                   'Air_Temperature_(°C)',
                                                   'Relative_Humidity_(%)',
                                                   'Visibility_(m)',
                                                   'Total_Cloud_Cover_(oktas)',
                                                   'Snow_Height_(cm)',
                                                   'Rainfall_(3h,_mm)',
                                                   'Rainfall_(12h,_mm)',
                                                   'month', 'weekday', 'day',
                                                   'hour']),
                                                 ('passthrough', 'passthrough',
                                                  ['year', 'is_weekend',
                                                   'is_school_holiday',
                                                   'road_work', 'confinement',
                                                   'couvre_feu'])])),
                ('model',
                 RandomForestRegressor(max_depth=5, min_samples_leaf=5,
                                       min_samples_split=10, n_estimators=50,
                                       random_state=42))])

In [11]:
submission = utils.test_fit_and_submission(X_test, pipeline)

In [15]:
! git push origin main

Enumerating objects: 4, done.
Counting objects: 100% (4/4), done.
Delta compression using up to 8 threads
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 410 bytes | 410.00 KiB/s, done.
Total 3 (delta 1), reused 0 (delta 0), pack-reused 0 (from 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/felixbroch/Python_Data_Challenge.git
   96afe37..c9e7565  main -> main
